In [1]:
import os

os.chdir('../')

In [2]:
import pandas as pd
from pyspark.ml.feature import StringIndexer, OneHotEncoder
import plotly.express as px
from Utils.SparkSessionManager import SparkSessionManager
from Service.DataReader import DataReader
from Service.VMAnalyzer import VMAnalyzer
import json


spark = SparkSessionManager.create_session()
dt_reader = DataReader(spark)
with open('settings.json', 'r') as file:
    data = json.load(file)

local_settings = data['Local']
filepath = local_settings['filepath']
filename = local_settings['output']
output = local_settings['output']

csv_file, vmSchema = dt_reader.read_data(filepath, filename, output)
dt_analyzer = VMAnalyzer(spark, csv_file, vmSchema)
dataFrame = dt_analyzer.get_df()
#dt_analyzer.check_correlation(processed_df, "baseScore", "exploitabilityScore")
#dt_analyzer.plot_correlation(processed_df, "baseScore", "exploitabilityScore", 700, 600)
dt_analyzer.find_mode_category(dataFrame, "attackVector", "baseScore", "Attack Vector", "Base Score", 700, 600)



+----------------+----------+
|   Attack Vector|Base Score|
+----------------+----------+
|ADJACENT_NETWORK|       6.5|
|           LOCAL|       7.8|
|         NETWORK|       9.8|
|        PHYSICAL|       4.6|
+----------------+----------+

